In [1]:
!pip install -U h2o

     |████████████████████████████████| 123.7MB 58kB/s 
  Created wheel for h2o: filename=h2o-3.26.0.10-py2.py3-none-any.whl size=123719583 sha256=e4ce377ad20fee5a105a6a672ceab0d59639fe00df9f5f3e9260f7fd729ef0a2
  Stored in directory: /tmp/.cache/pip/wheels/7d/09/bc/23eaa75a5948deb1d6c88d9ee3a20f94bac7d9df6c55ec8aa3
Successfully built h2o
  Found existing installation: h2o 3.26.0.8
    Uninstalling h2o-3.26.0.8:
      Successfully uninstalled h2o-3.26.0.8


In [2]:
import pandas as pd

import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_232"; OpenJDK Runtime Environment (build 1.8.0_232-8u232-b09-1~deb9u1-b09); OpenJDK 64-Bit Server VM (build 25.232-b09, mixed mode)
  Starting server from /opt/conda/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp6q91564t
  JVM stdout: /tmp/tmp6q91564t/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp6q91564t/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.10
H2O cluster version age:,24 days
H2O cluster name:,H2O_from_python_unknownUser_aci743
H2O cluster total nodes:,1
H2O cluster free memory:,3.556 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


Of course, we can analyse data, do model selection, fine-tune hyperparams. But what about something else ...

In [3]:
train = h2o.import_file('/kaggle/input/infopulsehackathon/train.csv')
test = h2o.import_file('/kaggle/input/infopulsehackathon/test.csv')

sample_submission = pd.read_csv('/kaggle/input/infopulsehackathon/sample_submission.csv')

y = "Energy_consumption"
x = list(train.columns) 
x.remove(y)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


# Magic

In [4]:
aml = H2OAutoML(max_runtime_secs = 18000, sort_metric='mse', stopping_metric='MSE' , stopping_rounds=100)
aml.train(x = x, y = y, training_frame = train)

aml.leaderboard

AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20191201_195713,620670,787.826,620670,537.723,0.372911
StackedEnsemble_BestOfFamily_AutoML_20191201_195713,626571,791.562,626571,543.184,0.377761
GBM_grid_1_AutoML_20191201_195713_model_3,649743,806.066,649743,553.989,0.385169
XGBoost_grid_1_AutoML_20191201_195713_model_3,660415,812.659,660415,561.569,0.387243
XGBoost_grid_1_AutoML_20191201_195713_model_57,661533,813.347,661533,557.686,0.386083
XGBoost_grid_1_AutoML_20191201_195713_model_54,662851,814.156,662851,558.391,0.387735
XGBoost_grid_1_AutoML_20191201_195713_model_60,662909,814.192,662909,555.716,0.381891
XGBoost_grid_1_AutoML_20191201_195713_model_55,663465,814.534,663465,559.115,0.384718
XGBoost_grid_1_AutoML_20191201_195713_model_5,664336,815.068,664336,562.87,0.389085
XGBoost_grid_1_AutoML_20191201_195713_model_1,664622,815.243,664622,559.191,0.386519


# Some more magic

In [5]:
sample_submission['Energy_consumption'] = aml.leader.predict(test).as_data_frame()['predict']
sample_submission.loc[sample_submission['Energy_consumption'] < 0, 'Energy_consumption'] = 0

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [6]:
sample_submission[['Id','Energy_consumption']].to_csv('submission.csv', index=False)

# Final thougths

You can GO DEEPER! or ANALYSE DATA! But you can just blend AutoMLs. According to [this discussion](https://www.kaggle.com/c/infopulsehackathon/discussion/119729#latest-685093) will it dare it? ;)